<a href="https://colab.research.google.com/github/torimonn/disclosure/blob/main/main_extraction_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 財務諸表抽出ノートブック
このノートブックでは、Google Generative AI (Gemini) を用いてPDFから
貸借対照表と損益計算書を抽出し、CSVとして保存する処理を解説付きで実行します。

## 必要なライブラリのインストールとインポート
Colab 環境では下記のようにライブラリをインストールしてから実行してください。

In [10]:
# ======================================================================
# ステップ1：準備 ～ 必要な道具（ライブラリ）を用意する
# ======================================================================
# プログラムを作るには、便利な機能が詰まった「ライブラリ」という道具箱が必要です。
# 車を組み立てるのに、エンジンやタイヤを外部から持ってくるのと同じイメージです。
print("ステップ1: 必要なライブラリをインストールします...")
# `!pip install` はColabに「このライブラリを追加して」とお願いするコマンドです。
# -qU: インストール中のメッセージを静かにし(-q)、もし古ければ最新版に更新します(-U)。
!pip install -qU google-genai pydantic pypdf pandas

# --- ライブラリから、実際に使う機能を取り出す ---
# これから使う機能を、それぞれの道具箱から取り出して、いつでも使えるように準備します。
import os  # ファイルの場所(パス)や名前を扱うための道具
import json  # AIとのデータ形式(JSON)を扱うための道具
import re  # 文字列の中から特定のパターンを見つけるための道具（正規表現）
from datetime import date  # 日付(年-月-日)を扱うための道具
from typing import List, Optional, Union, Tuple  # プログラムの可読性を上げるための型ヒント機能
from functools import reduce # 複数のデータを順番に処理するために使う便利な道具

# ★今回の主役級ライブラリたち★
import pandas as pd  # Excelのような表データを扱うプロ。CSV作成の達人。'pd'というあだ名で呼びます。
import google.generativeai as genai  # GoogleのAI(Gemini)と会話するための中心的な道具。'genai'と呼びます。
from pydantic import BaseModel, Field, ValidationError, field_validator  # AIに出力の「完璧な設計図」を渡すための専門家。
from pypdf import PdfReader  # PDFファイルから文字を読み取るための道具。
from google.colab import userdata, files  # Google Colabの特別な機能（APIキー管理、ファイル操作）を使うための道具。

print("ライブラリの準備が完了しました。\n")


ステップ1: 必要なライブラリをインストールします...
ライブラリの準備が完了しました。



In [11]:
# ======================================================================
# ステップ2：認証 ～ AIと話すための合鍵を設定する
# ======================================================================
# AIサービスを使うには、「APIキー」という自分専用の合鍵が必要です。
# このキーをコードに直接書くと世界中の人に見られてしまう危険があるため、
# Colabの安全な金庫「シークレット」から読み込みます。
print("ステップ2: Google AIのAPIキーを設定します...")
# `try...except`は、「もしエラーが起きても、パニックにならずにこの処理をしてね」という安全装置です。
try:
    # Colabの金庫(userdata)から 'GOOGLE_API_KEY' という名前の鍵を取り出します。
    # ※事前にColabの左側メニュー(鍵マーク)からAPIキーを登録しておく必要があります。
    google_api_key = userdata.get('GOOGLE_API_KEY')
    # 取り出した鍵を、Google AIの道具箱(genai)にセットします。これでAIと話せるようになります。
    genai.configure(api_key=google_api_key)
    print('APIキーの設定が完了しました。\n')
except userdata.SecretNotFoundError as e:
    # もし金庫に鍵がなかった場合のエラー処理
    print(f"エラー: Colabのシークレットに 'GOOGLE_API_KEY' が見つかりません。: {e}")
    print("対処法: Google Colabの左メニューにある鍵のアイコンから 'GOOGLE_API_KEY' を設定してください。")
    raise SystemExit  # 致命的なエラーなのでプログラムを終了します。
except Exception as e:
    # その他の予期せぬエラーが起きた場合の処理
    print(f'APIキー設定中に予期せぬエラーが発生しました: {e}')
    raise SystemExit


ステップ2: Google AIのAPIキーを設定します...
APIキーの設定が完了しました。



In [12]:
# ======================================================================
# ステップ3：ファイル受付 ～ 処理したいPDFをアップロードする
# ======================================================================
print("ステップ3: 処理したいPDFファイルを1つ以上アップロードしてください...")
# このコードを実行すると、ファイル選択ボタンが表示されます。
uploaded = files.upload()
# アップロードされたファイルの名前から、プログラムがアクセスできる場所のリストを作成します。
# 例: 'my_report.pdf' という名前 -> '/content/my_report.pdf' というPCが理解できる住所に変換
pdf_paths = [os.path.join('/content', name) for name in uploaded.keys()]
print(f"\nアップロードされたPDFファイル: {pdf_paths}\n")


ステップ3: 処理したいPDFファイルを1つ以上アップロードしてください...


Saving disclosure_2024.pdf to disclosure_2024 (2).pdf
Saving disclo2024_0.pdf to disclo2024_0 (2).pdf
Saving all.pdf to all (2).pdf
Saving tamashin_disclo2024.pdf to tamashin_disclo2024 (2).pdf
Saving disclo24.pdf to disclo24 (2).pdf
Saving disclo_2024keisu.pdf to disclo_2024keisu (2).pdf

アップロードされたPDFファイル: ['/content/disclosure_2024 (2).pdf', '/content/disclo2024_0 (2).pdf', '/content/all (2).pdf', '/content/tamashin_disclo2024 (2).pdf', '/content/disclo24 (2).pdf', '/content/disclo_2024keisu (2).pdf']



In [13]:
# ======================================================================
# ステップ4：AIモデルの指定 ～ どのAIにお願いするか決める
# ======================================================================
print("ステップ4: 使用するAIモデルを指定します...")
# ここではGoogleの高速・高性能なモデル 'gemini-2.5-flash-preview-05-20' を使用します。
MODEL_NAME = "gemini-2.5-flash-preview-05-20"
print(f"モデル '{MODEL_NAME}' を使用します。\n")

ステップ4: 使用するAIモデルを指定します...
モデル 'gemini-2.5-flash-preview-05-20' を使用します。



In [14]:
# ======================================================================
# ★★★【このコードの最重要改善点 その１】★★★
# ステップ5：勘定科目の「お手本（テンプレート）」を定義する
# ======================================================================
# 問題点：AIはPDFから見つけた順にデータを返してくるため、会社ごとに科目の並び順がバラバラになってしまう。
# 解決策：ここで、最終的なCSVの行の並び順を決定する「完璧なお手本」となるリストを定義します。
#         このリストの順番が、そのまま出力されるCSVの科目順になります。
print("ステップ5: 標準的な勘定科目の並び順テンプレートを定義します...")

KANJO_KAMOKU_TEMPLATE = [
    # 貸借対照表 - 資産の部
    '-- 貸借対照表 --',
    '資産の部',
    '資産の部 - 現金',
    '資産の部 - 現金及び預け金',
    '資産の部 - 現預金',
    '資産の部 - 現金預け金',
    '資産の部 - 買入金銭債権',
    '資産の部 - 特定取引資産',
    '資産の部 - 商品有価証券',
    '資産の部 - 金銭の信託',
    '資産の部 - 有価証券',
    '資産の部 - 有価証券 - 国債',
    '資産の部 - 有価証券 - 地方債',
    '資産の部 - 有価証券 - 社債',
    '資産の部 - 有価証券 - 株式',
    '資産の部 - 有価証券 - その他の証券',
    '資産の部 - 貸出金',
    '資産の部 - 貸出金 - 手形貸付',
    '資産の部 - 貸出金 - 証書貸付',
    '資産の部 - 貸出金 - 割引手形',
    '資産の部 - 貸出金 - 当座貸越',
    '資産の部 - 外国為替',
    '資産の部 - その他資産',
    '資産の部 - その他資産 - 未決済為替貸',
    '資産の部 - その他資産 - 未収収益',
    '資産の部 - その他資産 - その他の資産',
    '資産の部 - 有形固定資産',
    '資産の部 - 有形固定資産 - 建物',
    '資産の部 - 有形固定資産 - 土地',
    '資産の部 - 有形固定資産 - リース資産',
    '資産の部 - 有形固定資産 - 建設仮勘定',
    '資産の部 - 有形固定資産 - その他の有形固定資産',
    '資産の部 - 無形固定資産',
    '資産の部 - 無形固定資産 - ソフトウェア',
    '資産の部 - 無形固定資産 - リース資産',
    '資産の部 - 無形固定資産 - その他の無形固定資産',
    '資産の部 - 繰延税金資産',
    '資産の部 - 支払承諾見返',
    '資産の部 - 貸倒引当金',
    '資産の部合計',
    # 貸借対照表 - 負債の部
    '負債の部',
    '負債の部 - 預金積金',
    '負債の部 - 預金積金 - 当座預金',
    '負債の部 - 預金積金 - 普通預金',
    '負債の部 - 預金積金 - 定期預金',
    '負債の部 - 預金積金 - 貯蓄預金',
    '負債の部 - 預金積金 - 通知預金',
    '負まあ、の部 - 預金積金 - 定期預金',
    '負債の部 - 預金積金 - 定期積金',
    '負債の部 - 預金積金 - その他の預金',
    '負債の部 - 譲渡性預金',
    '負債の部 - 借用金',
    '負債の部 - 特定取引負債',
    '負債の部 - 外国為替',
    '負債の部 - その他負債',
    '負債の部 - 賞与引当金',
    '負債の部 - 役員賞与引当金',
    '負債の部 - 退職給付引当金',
    '負債の部 - 役員退職慰労引当金',
    '負債の部 - 睡眠預金払戻損失引当金',
    '負債の部 - 偶発損失引当金',
    '負債の部 - 繰延税金負債',
    '負債の部 - 支払承諾',
    '負債の部合計',
    # 貸借対照表 - 純資産の部
    '純資産の部',
    '純資産の部 - 出資金',
    '純資産の部 - 資本剰余金',
    '純資産の部 - 利益剰余金',
    '純資産の部 - 利益剰余金 - 利益準備金',
    '純資産の部 - 利益剰余金 - その他利益剰余金',
    '純資産の部 - 自己株式',
    '純資産の部 - 評価・換算差額等',
    '純資産の部 - 評価・換算差額等 - その他有価証券評価差額金',
    '純資産の部 - 評価・換算差額等 - 繰延ヘッジ損益',
    '純資産の部 - 評価・換算差額等 - 土地再評価差額金',
    '純資産の部合計',
    '負債純資産合計',
    '', # CSVに見やすいように空行を入れるための目印
    # 損益計算書
    '-- 損益計算書 --',
    '経常収益',
    '経常収益 - 資金運用収益',
    '経常収益 - 役務取引等収益',
    '経常収益 - その他業務収益',
    '経常収益 - その他経常収益',
    '経常費用',
    '経常費用 - 資金調達費用',
    '経常費用 - 役務取引等費用',
    '経常費用 - その他業務費用',
    '経常費用 - 経費',
    '経常費用 - その他経常費用',
    '経常利益',
    '特別利益',
    '特別損失',
    '税引前当期純利益',
    '法人税、住民税及び事業税',
    '法人税等調整額',
    '当期純利益',
]
print("テンプレートの準備が完了しました。\n")


ステップ5: 標準的な勘定科目の並び順テンプレートを定義します...
テンプレートの準備が完了しました。



In [15]:
# ======================================================================
# ステップ6：データ構造の設計図を定義する (Pydanticモデル)
# ======================================================================
# AIに正確な仕事をしてもらうための「詳細な設計図」を定義します。
print("ステップ6: AIへの指示に必要なデータ構造の設計図を準備します...")
class PDFMetadata(BaseModel):
    """PDF全体の基本情報（会社名、ページ番号など）の設計図"""
    company_name_japanese: str = Field(description="会社名（日本語）。例：〇〇信用金庫")
    company_name_english: Optional[str] = Field(default=None, description="会社名（英語）。なければ省略可")
    balance_sheet_pages_1_indexed: List[int] = Field(description="貸借対照表が記載されているページの番号リスト。例: [10, 11]")
    income_statement_pages_1_indexed: Optional[List[int]] = Field(default=None, description="損益計算書が記載されているページの番号リスト。なければ省略可")
    balance_sheet_amount_unit: int = Field(description="貸借対照表の金額単位。例：表に「単位：百万円」とあれば 1000000 を返す。なければ「単位：円」")
    income_statement_amount_unit: Optional[int] = Field(default=None, description="損益計算書の金額単位。貸借対照表と同じなら省略可")

class StatementItem(BaseModel):
    """財務諸表の各項目（「現金及び預金」など）一つの設計図"""
    name_japanese: str
    name_english: str
    value: Optional[Union[int, float, str]]
    indent_level: int
    children: List['StatementItem'] = Field(default_factory=list)

    @field_validator('value', mode='before')
    @classmethod
    def clean_value(cls, v):
        if v is None or isinstance(v, (int, float)): return v
        if isinstance(v, str):
            s = v.strip()
            if not s: return None
            s = s.replace('△', '-').replace('▲', '-').replace(',', '')
            if s.startswith('(') and s.endswith(')'): s = '-' + s[1:-1]
            try: return float(s)
            except (ValueError, TypeError): return None
        return v
StatementItem.model_rebuild()

class Section(BaseModel):
    """「資産の部」や「負債の部」といった、大きなかたまりの設計図"""
    items: List[StatementItem]
    total_value: Optional[Union[int, float, str]] = Field(default=None)
    @field_validator('total_value', mode='before')
    @classmethod
    def clean_total_value(cls, v):
        if v is None or isinstance(v, (int, float)): return v
        if isinstance(v, str):
            s = v.strip().replace('△', '-').replace('▲', '-').replace(',', '')
            if s.startswith('(') and s.endswith(')'): s = '-' + s[1:-1]
            try: return float(s)
            except (ValueError, TypeError): return None
        return v

class FiscalYearBalanceSheet(BaseModel):
    end_date: date
    assets: Section
    liabilities: Section
    net_assets: Section

class BalanceSheetBody(BaseModel):
    fiscal_year_data: List[FiscalYearBalanceSheet]

class BalanceSheetResponse(BaseModel):
    balance_sheet: BalanceSheetBody

class FiscalYearIncomeStatement(BaseModel):
    end_date: date
    items: List[StatementItem]

class IncomeStatementBody(BaseModel):
    fiscal_year_data: List[FiscalYearIncomeStatement]

class IncomeStatementResponse(BaseModel):
    income_statement: IncomeStatementBody

print("設計図の準備が完了しました。\n")

ステップ6: AIへの指示に必要なデータ構造の設計図を準備します...
設計図の準備が完了しました。



In [16]:
# ======================================================================
# ステップ7：補助的な機能（関数）を定義する
# ======================================================================
print("ステップ7: プログラムで使う補助的な機能（関数）を定義します...")
# (upload_pdf_to_genai, extract_text_from_pdf_pypdf, create_generative_content_parts, call_llm_for_structured_output は変更なし)
def upload_pdf_to_genai(pdf_file_path: str) -> Optional[genai.protos.File]:
    try:
        print(f"  - PDF '{os.path.basename(pdf_file_path)}' をAIサーバーにアップロード中...")
        return genai.upload_file(pdf_file_path)
    except Exception as e:
        print(f"  - エラー: PDFのアップロードに失敗: {e}")
        return None

def extract_text_from_pdf_pypdf(pdf_file_path: str, pages_0_indexed: Optional[List[int]]) -> str:
    try:
        reader = PdfReader(pdf_file_path)
        pages = pages_0_indexed if pages_0_indexed is not None else range(len(reader.pages))
        texts = [reader.pages[p].extract_text() or "" for p in pages]
        return "\n".join(texts)
    except Exception as e:
        print(f"  - エラー: PDFからのテキスト抽出に失敗: {e}")
        return ""

def create_generative_content_parts(instruction: str, uploaded_file: genai.protos.File, aux_text: Optional[str] = None) -> List[Union[str, genai.protos.File]]:
    parts = [instruction, uploaded_file]
    if aux_text:
        parts.append("\n\n### 補助テキスト情報:\n" + aux_text)
    return parts

def call_llm_for_structured_output(content_parts: List[Union[str, genai.protos.File]], output_model: BaseModel, max_retries: int = 1) -> Optional[BaseModel]:
    current_content_parts = list(content_parts)
    attempt = 0
    while attempt <= max_retries:
        attempt += 1
        print(f"    - AI呼び出し試行: {attempt}回目")

        response_text_for_debug = ""
        try:
            model = genai.GenerativeModel(model_name=MODEL_NAME)
            response = model.generate_content(contents=current_content_parts, generation_config=genai.GenerationConfig(response_mime_type="application/json", temperature=0))
            response_text_for_debug = response.text
            data = json.loads(response_text_for_debug)
            return output_model.model_validate(data)
        except (ValidationError, json.JSONDecodeError) as e:
            print(f"    - エラー (試行 {attempt}回目): AIの応答の解析に失敗。詳細: {e}")
            if attempt > max_retries:
                print("    - リトライ上限到達。最終エラー応答を出力します。")
                print(f"---AIからの最終エラー応答---\n{response_text_for_debug}\n-------------------------")
                return None

            print("    - AIに自己修正を促してリトライします...")
            error_feedback = f"### 追加指示:\nあなたの前回のアウトプットはエラーで解析できませんでした。\nエラー内容: {e}\n問題のあったJSON:\n```json\n{response_text_for_debug}\n```\nこのエラーを修正し、**完全に有効なJSON**のみを出力してください。"
            current_content_parts.append(error_feedback)
        except Exception as e:
            print(f"    - 予期せぬエラー (試行 {attempt}回目): AI呼び出し中に問題発生: {e}")
            return None
    return None

def multiply_unit(items: List[StatementItem], amount_unit: int):
    for item in items:
        if item.value is not None and isinstance(item.value, (int, float)):
            item.value *= amount_unit
        if item.children:
            multiply_unit(item.children, amount_unit)

# (flatten_and_build_dict と build_financial_data_dict は前回から変更なし)
def flatten_and_build_dict(items: List[StatementItem], parent_prefix: str) -> dict:
    output_dict = {}
    for item in items:
        unique_key = f"{parent_prefix} - {item.name_japanese}" if parent_prefix else item.name_japanese
        if '合計' in item.name_japanese or '計' in item.name_japanese:
            continue
        output_dict[unique_key] = item.value if item.value is not None else ''
        if item.children:
            child_dict = flatten_and_build_dict(item.children, unique_key)
            output_dict.update(child_dict)
    return output_dict

def build_financial_data_dict(bs_resp: BalanceSheetResponse, pl_resp: IncomeStatementResponse, metadata: PDFMetadata) -> dict:
    """抽出した全データから、最終的なCSVの1社分のデータ辞書を作成する関数"""

    # --- ここが最重要ポイント！ ---
    # AIが返した複数の決算期データの中から、end_dateが最新のものを選択する
    # `max`関数と`key=lambda`を使って、リスト内のオブジェクトの日付を比較し、最新のものを見つけます。
    if not bs_resp.balance_sheet.fiscal_year_data:
        print("  - エラー: 貸借対照表の決算期データが見つかりません。")
        return {} # 空の辞書を返す
    latest_bs = max(bs_resp.balance_sheet.fiscal_year_data, key=lambda x: x.end_date)

    if not pl_resp.income_statement.fiscal_year_data:
        print("  - エラー: 損益計算書の決算期データが見つかりません。")
        return {} # 空の辞書を返す
    latest_pl = max(pl_resp.income_statement.fiscal_year_data, key=lambda x: x.end_date)

    print(f"  - 直近決算期として {latest_bs.end_date.strftime('%Y-%m-%d')} のデータを選択しました。")
    # --------------------------

    bs_unit = metadata.balance_sheet_amount_unit
    pl_unit = metadata.income_statement_amount_unit or bs_unit

    multiply_unit(latest_bs.assets.items, bs_unit)
    multiply_unit(latest_bs.liabilities.items, bs_unit)
    multiply_unit(latest_bs.net_assets.items, bs_unit)
    multiply_unit(latest_pl.items, pl_unit)

    final_data = {}
    final_data.update(flatten_and_build_dict(latest_bs.assets.items, "資産の部"))
    final_data.update(flatten_and_build_dict(latest_bs.liabilities.items, "負債の部"))
    final_data.update(flatten_and_build_dict(latest_bs.net_assets.items, "純資産の部"))

    if latest_bs.assets.total_value is not None: final_data['資産の部合計'] = latest_bs.assets.total_value * bs_unit
    if latest_bs.liabilities.total_value is not None: final_data['負債の部合計'] = latest_bs.liabilities.total_value * bs_unit
    if latest_bs.net_assets.total_value is not None:
        final_data['純資産の部合計'] = latest_bs.net_assets.total_value * bs_unit
        if latest_bs.liabilities.total_value is not None:
             final_data['負債純資産合計'] = (latest_bs.liabilities.total_value + latest_bs.net_assets.total_value) * bs_unit

    final_data.update(flatten_and_build_dict(latest_pl.items, ""))

    # メタデータも最新の決算日に合わせて辞書に追加
    final_data['金庫名'] = metadata.company_name_japanese
    final_data['決算期'] = latest_bs.end_date.strftime('%Y-%m-%d')
    final_data['貸借対照表記載ページ'] = ','.join(map(str, metadata.balance_sheet_pages_1_indexed))
    final_data['損益計算書記載ページ'] = ','.join(map(str, metadata.income_statement_pages_1_indexed or []))

    return final_data

print("補助的な機能（関数）の定義が完了しました。\n")


ステップ7: プログラムで使う補助的な機能（関数）を定義します...
補助的な機能（関数）の定義が完了しました。



In [17]:
# ======================================================================
# ステップ8：司令塔となる関数を定義する
# ======================================================================
# これまで定義した全部品を使って、1つのPDFファイルを処理する一連の流れを実行する関数です。
print("ステップ8: 全体の処理の流れを定義する司令塔関数を準備します...")

def process_pdf(pdf_path: str) -> Optional[dict]:
    """1つのPDFを処理し、{勘定科目: 金額} の辞書を返す司令塔"""
    print(f"\n処理開始: {pdf_path}")
    uploaded_file = upload_pdf_to_genai(pdf_path)
    if not uploaded_file: return None

    try:
        # --- ステップA: メタデータ抽出 ---
        print("  - ステップA: 基本情報を抽出中...")
        meta_schema_json = json.dumps(PDFMetadata.model_json_schema(), ensure_ascii=False, indent=2)
        meta_instruction = "PDFから会社名、貸借対照表ページ、損益計算書ページ、金額単位を抽出してください。出力はJSONのみで、スキーマは次の通りです:\n" + meta_schema_json
        meta_parts = create_generative_content_parts(meta_instruction, uploaded_file)
        metadata: Optional[PDFMetadata] = call_llm_for_structured_output(meta_parts, PDFMetadata)
        if not metadata: return None

        # --- ステップB: 貸借対照表(B/S)抽出 ---
        print("  - ステップB: 貸借対照表のデータを抽出中...")
        pages_bs = [p - 1 for p in metadata.balance_sheet_pages_1_indexed]
        aux_text_bs = extract_text_from_pdf_pypdf(pdf_path, pages_bs)
        bs_schema_json = json.dumps(BalanceSheetResponse.model_json_schema(), ensure_ascii=False, indent=2)
        # AIへの指示に「直近決算期」という条件を追加
        bs_instruction = f"""会社名「{metadata.company_name_japanese}」の単体貸借対照表を抽出し、次のJSONスキーマに**厳密に**従ってください。
- **PDF内に複数の決算期（例：当期と前期）がある場合、必ず最新（直近）の決算期のデータのみを抽出してください。**
- **スキーマに定義されていないキーは絶対に追加しないでください。**
- **全ての金額(value, total_value)は、マイナス記号(△, ▲, -)を含めて必ず数値(number)型で出力してください。**
- 勘定科目の階層構造を `indent_level` と `children` で正しく表現してください。

スキーマ:
{bs_schema_json}"""
        bs_parts = create_generative_content_parts(bs_instruction, uploaded_file, aux_text_bs)
        bs_resp: Optional[BalanceSheetResponse] = call_llm_for_structured_output(bs_parts, BalanceSheetResponse)
        if not bs_resp: return None

        # --- ステップC: 損益計算書(P/L)抽出 ---
        print("  - ステップC: 損益計算書のデータを抽出中...")
        pages_pl = [p - 1 for p in metadata.income_statement_pages_1_indexed] if metadata.income_statement_pages_1_indexed else None
        aux_text_pl = extract_text_from_pdf_pypdf(pdf_path, pages_pl)
        pl_schema_json = json.dumps(IncomeStatementResponse.model_json_schema(), ensure_ascii=False, indent=2)
        # AIへの指示に「直近決算期」という条件を追加
        pl_instruction = f"""会社名「{metadata.company_name_japanese}」の単体損益計算書を抽出し、次のJSONスキーマに**厳密に**従ってください。
- **PDF内に複数の決算期（例：当期と前期）がある場合、必ず最新（直近）の決算期のデータのみを抽出してください。**
- **スキーマに定義されていないキーは絶対に追加しないでください。**
- **全ての金額(value)は、マイナス記号(△, ▲, -)を含めて必ず数値(number)型で出力してください。**
- 勘定科目の階層構造を `indent_level` と `children` で正しく表現してください。

スキーマ:
{pl_schema_json}"""
        pl_parts = create_generative_content_parts(pl_instruction, uploaded_file, aux_text_pl)
        pl_resp: Optional[IncomeStatementResponse] = call_llm_for_structured_output(pl_parts, IncomeStatementResponse)
        if not pl_resp: return None

        # --- ステップD: データ辞書の生成 ---
        print("  - ステップD: 抽出結果を整形中...")
        financial_dict = build_financial_data_dict(bs_resp, pl_resp, metadata)

        print(f"処理完了: {pdf_path}")
        return financial_dict

    finally:
        try:
            genai.delete_file(uploaded_file.name)
            print(f"  - 一時ファイル '{uploaded_file.name}' をサーバーから削除しました。")
        except Exception as e:
            print(f"  - 警告: 一時ファイルの削除に失敗: {e}")

print("司令塔関数の準備が完了しました。\n")


ステップ8: 全体の処理の流れを定義する司令塔関数を準備します...
司令塔関数の準備が完了しました。



In [18]:
# ======================================================================
# ステップ9：メイン処理 ～ 全てのPDFを順番に処理する
# ======================================================================
print("ステップ9: メイン処理を開始します...")
# 各PDFから抽出した辞書データ（{科目: 金額, ...}）を、このリストに格納していきます。
all_financial_data = []

# アップロードされたPDFファイルのパスを一つずつ取り出して、司令塔(process_pdf)に渡します。
for path in pdf_paths:
    result_dict = process_pdf(path)

    # 司令塔が正しく辞書データを返した場合のみ、リストに追加します。
    if result_dict is not None:
        all_financial_data.append(result_dict)

ステップ9: メイン処理を開始します...

処理開始: /content/disclosure_2024 (2).pdf
  - PDF 'disclosure_2024 (2).pdf' をAIサーバーにアップロード中...
  - ステップA: 基本情報を抽出中...
    - AI呼び出し試行: 1回目
  - ステップB: 貸借対照表のデータを抽出中...
    - AI呼び出し試行: 1回目
  - ステップC: 損益計算書のデータを抽出中...
    - AI呼び出し試行: 1回目
  - ステップD: 抽出結果を整形中...
処理完了: /content/disclosure_2024 (2).pdf
  - 一時ファイル 'files/hkllota6v2f8' をサーバーから削除しました。

処理開始: /content/disclo2024_0 (2).pdf
  - PDF 'disclo2024_0 (2).pdf' をAIサーバーにアップロード中...
  - ステップA: 基本情報を抽出中...
    - AI呼び出し試行: 1回目
  - ステップB: 貸借対照表のデータを抽出中...
    - AI呼び出し試行: 1回目
  - ステップC: 損益計算書のデータを抽出中...
    - AI呼び出し試行: 1回目
  - ステップD: 抽出結果を整形中...
処理完了: /content/disclo2024_0 (2).pdf
  - 一時ファイル 'files/vbsww6ox5ywf' をサーバーから削除しました。

処理開始: /content/all (2).pdf
  - PDF 'all (2).pdf' をAIサーバーにアップロード中...
  - ステップA: 基本情報を抽出中...
    - AI呼び出し試行: 1回目
  - ステップB: 貸借対照表のデータを抽出中...
    - AI呼び出し試行: 1回目
  - ステップC: 損益計算書のデータを抽出中...
    - AI呼び出し試行: 1回目
  - ステップD: 抽出結果を整形中...
処理完了: /content/all (2).pdf
  - 一時ファイル 'files/j371uz2pc5ij' をサーバーから削除しました。


In [19]:
# ======================================================================
# ★★★【部門別ハイブリッド版】★★★
# ステップ10：最終仕上げ ～ テンプレートに合わせて整形しCSVに出力
# ======================================================================
print("\nステップ10: 最終的なCSVファイルの作成を開始します...")
if all_financial_data:
    print("  - 全ての会社のデータを一つの大きな表に変換中...")
    df = pd.DataFrame(all_financial_data).set_index('金庫名')
    df = df.T # 行と列を入れ替え、科目を行名、会社名を列名にする

    print("  - 勘定科目を部門ごとに分類し、並べ替えの準備中...")
    # --- ここからが部門別ハイブリッド処理の核心 ---
    # 抽出された全科目を部門ごとに分類するためのリストを準備します。
    all_items_in_df = set(df.index) # 抽出されたユニークな科目名全部
    bs_prefixes = ("資産の部", "負債の部", "純資産の部")

    # 部門ごとに「テンプレート内の項目」と「テンプレート外の項目」を仕分けします。
    final_ordered_list = []

    # 1. メタデータ部分の処理
    meta_keys = ['決算期', '貸借対照表記載ページ', '損益計算書記載ページ']
    final_ordered_list.extend(meta_keys)

    # 2. 貸借対照表部分の処理
    final_ordered_list.append('-- 貸借対照表 --')
    # テンプレートからB/S関連の科目だけを抜き出します。
    bs_template_items = [item for item in KANJO_KAMOKU_TEMPLATE if item.startswith(bs_prefixes) or '合計' in item]
    # B/S関連科目で、テンプレートにあるものを順に追加します。
    final_ordered_list.extend([item for item in bs_template_items if item in all_items_in_df])
    # B/S関連科目で、テンプレートにないものを抽出し、ソートして追加します。
    extra_bs_items = sorted([item for item in all_items_in_df if item.startswith(bs_prefixes) and item not in bs_template_items])
    final_ordered_list.extend(extra_bs_items)
    final_ordered_list.append('') # 空行

    # 3. 損益計算書部分の処理
    final_ordered_list.append('-- 損益計算書 --')
    # テンプレートからP/L関連の科目だけを抜き出します。
    pl_template_items = [item for item in KANJO_KAMOKU_TEMPLATE if not item.startswith(bs_prefixes) and item not in meta_keys and item not in ['-- 貸借対照表 --', '']]
    # P/L関連科目で、テンプレートにあるものを順に追加します。
    final_ordered_list.extend([item for item in pl_template_items if item in all_items_in_df])
    # P/L関連科目で、テンプレートにないものを抽出し、ソートして追加します。
    extra_pl_items = sorted([item for item in all_items_in_df if not item.startswith(bs_prefixes) and item not in pl_template_items and item not in meta_keys])
    final_ordered_list.extend(extra_pl_items)

    # 4. 最終的な並べ替え
    print("  - 最終的な並び順に従って表を整形中...")
    df_standardized = df.reindex(final_ordered_list)

    df_standardized.fillna('', inplace=True)
    df_standardized.reset_index(inplace=True)
    df_standardized.rename(columns={'index': '科目'}, inplace=True)

    output_filename = 'merged_financials_final.csv'
    df_standardized.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"\n🎉 成功: 全ての財務諸表を結合・整形し、'{output_filename}' として保存しました！")
    files.download(output_filename)
else:
    print("\nすべてのPDFの処理に失敗したため、出力するファイルはありません。")


ステップ10: 最終的なCSVファイルの作成を開始します...
  - 全ての会社のデータを一つの大きな表に変換中...
  - 勘定科目を部門ごとに分類し、並べ替えの準備中...
  - 最終的な並び順に従って表を整形中...

🎉 成功: 全ての財務諸表を結合・整形し、'merged_financials_final.csv' として保存しました！


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>